In [1]:
import json 
import pandas as pd

In [2]:
with open ("Lyrics_JohnMayer.json") as file:
    
    data = json.load(file)
    
    master_list = []
    
    for row in data['songs']:
        
        temp_dict = {}
        
        try: 
            temp_dict["title"] = row["title"]
        except:
            temp_dict["title"] = None
        
        try: 
            temp_dict["release_date"] = row["release_date"]
        except: 
            temp_dict["release_date"] = None
            
        try: 
            temp_dict["album"] = row["album"]["name"]
        except: 
            temp_dict["album"] = None
            
        try:     
            temp_dict["lyrics"] = row["lyrics"]
        except: 
            temp_dict["lyrics"] = None
        
        master_list.append(temp_dict)

In [3]:
df = pd.DataFrame(master_list)

In [4]:
pd.set_option('display.max_rows', None)
#df

In [5]:
df.isnull().sum()

title             0
release_date    126
album            94
lyrics            0
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
#df.reset_index(drop=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109 entries, 0 to 208
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         109 non-null    object
 1   release_date  109 non-null    object
 2   album         109 non-null    object
 3   lyrics        109 non-null    object
dtypes: object(4)
memory usage: 4.3+ KB


In [9]:
df['release_date'] = pd.to_datetime(df.release_date, format='%Y-%m-%d')

In [10]:
df["year"] = df["release_date"].dt.strftime('%Y')

In [11]:
# df.head()

,title,release_date,album,lyrics,year
0,New Light,2018-05-10,Sob Rock,"New Light Lyrics[Intro]\nAh, ah, ah\nAh...\n\n...",2018
1,Gravity,2007-01-31,Continuum,Gravity Lyrics[Chorus]\nGravity is working aga...,2007
2,Slow Dancing in a Burning Room,2006-09-12,Continuum,Slow Dancing in a Burning Room Lyrics[Verse 1]...,2006
3,Daughters,2004-09-28,Heavier Things,Daughters Lyrics[Verse 1]\nI know a girl\nShe ...,2004
4,Free Fallin’ (Live),2008-07-01,Where The Light Is: John Mayer Live in Los Ang...,Free Fallin’ (Live) Lyrics[Verse 1]\nShe's a g...,2008


In [12]:
# Step 1: Find the most recent year for each album in the df data frame.

recent_year = df.groupby('album')['year'].max().reset_index()


In [13]:
# recent_year

,album,year
0,Any Given Thursday,2003
1,Battle Studies,2010
2,Born and Raised,2012
3,Continuum,2007
4,Heavier Things,2004
5,Inside Wants Out,1999
6,Paradise Valley,2013
7,Room For Squares [Japanese Edition],2003
8,Sob Rock,2021
9,The Search for Everything,2017


In [14]:
songs_by_album = df.groupby([df.album]).size().reset_index(name = 'count')

In [15]:
songs_by_album.sort_values(by='count', ascending=False).reset_index(drop=True)

,album,count
0,Born and Raised,13
1,Continuum,13
2,Room For Squares [Japanese Edition],13
3,Battle Studies,12
4,The Search for Everything,12
5,Paradise Valley,11
6,Sob Rock,10
7,Heavier Things,9
8,Inside Wants Out,5
9,Where The Light Is: John Mayer Live in Los Ang...,5


In [16]:
# Step 2: Merge the recent_year data frame with the songs_by_album data frame based on the 'album' column.

songs_by_album = songs_by_album.merge(recent_year, on='album')


In [17]:
# songs_by_album

,album,count,year
0,Any Given Thursday,4,2003
1,Battle Studies,12,2010
2,Born and Raised,13,2012
3,Continuum,13,2007
4,Heavier Things,9,2004
5,Inside Wants Out,5,1999
6,Paradise Valley,11,2013
7,Room For Squares [Japanese Edition],13,2003
8,Sob Rock,10,2021
9,The Search for Everything,12,2017


In [18]:
import altair as alt

In [ ]:
source = songs_by_album

# Create the chart
# Change the color of your 
bars = alt.Chart(source).mark_bar(color='#08F7DA').encode(
    x='count:Q',
    y=alt.Y("year:O", sort="-y"),
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=5,
    dy=-5,
    angle=5,
    fontSize = 15,
    color = '#F73708'
).encode(
    text='album'
)

(bars + text).properties(height=400, width=700)

In [43]:
bars.save('albums.html')

In [44]:
df.to_pickle("Mayer.pkl")